In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
modelo=pd.read_csv('data/modelo.csv',low_memory=False)

In [3]:
modelo.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
modelo.drop(['url', 'sku', 'skus','channel','staticpage','model','condition','storage','color','search_engine','device_type','campaign_source','country','region','search_term','new_vs_returning','city','region','screen_resolution','operating_system_version','browser_version'], axis=1, inplace=True)

In [5]:
training= pd.read_csv('data/training.csv',low_memory=False)

In [6]:
training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [7]:
len(modelo[modelo['event']=='conversion'])

7091

In [8]:
modelo[modelo['event']=='conversion']['person'].nunique()

4293

In [9]:
len(modelo[modelo['event']=='checkout'])

65315

In [10]:
modelo[modelo['event']=='checkout']['person'].nunique()

32833

# Armado del modelo

In [11]:
modelo.sort_values('person',inplace=True)

In [12]:
import calendar

modelo['timestamp'] = pd.to_datetime(modelo['timestamp'])

#modelo['dia']=modelo['timestamp'].dt.dayofyear
#modelo['dias_distintos']=modelo.groupby(['person','dia'])['person'].transform('count').astype(int)

modelo['dia']=modelo['timestamp'].dt.dayofyear
modelo['dias_distintos'] = modelo['person'].map(modelo.groupby(['person'])['dia'].nunique())   
modelo.drop('dia',1,inplace=True)

modelo['total_visitas_usuario'] = modelo.groupby('person')['person'].transform('count')


modelo['visitas_ultimo_mes']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month)==5]).size().unstack()[True])
modelo['visitas_ultimo_mes']=modelo['visitas_ultimo_mes'].fillna(0).astype(int)


modelo['visitas_ultima_semana']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True])
modelo['visitas_ultima_semana']=modelo['visitas_ultima_semana'].fillna(0).astype(int)


modelo['total_checkout']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['checkout'])
modelo['total_checkout']=modelo['total_checkout'].fillna(0).astype(int)


modelo['checkouts_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['checkout'])   
modelo['checkouts_ultimo_mes']=modelo['checkouts_ultimo_mes'].fillna(0).astype(int)


modelo['checkouts_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['checkout'])
modelo['checkouts_ultima_semana']=modelo['checkouts_ultima_semana'].fillna(0).astype(int)


modelo['total_compras']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['conversion'])
modelo['total_compras']=modelo['total_compras'].fillna(0).astype(int)


modelo['compras_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultimo_mes']=modelo['compras_ultimo_mes'].fillna(0).astype(int)


modelo['compras_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultima_semana']=modelo['compras_ultima_semana'].fillna(0).astype(int)


modelo['subscripciones_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['lead'])
modelo['subscripciones_ultimo_mes']=modelo['subscripciones_ultimo_mes'].fillna(0).astype(int)

In [14]:
modelo=modelo.drop_duplicates(subset=['person'])
modelo.drop(['timestamp','event'], 1, inplace=True)

In [15]:
modelo['total_checkout'].sum(), modelo['total_compras'].sum(),modelo['total_visitas_usuario'].sum(), len(modelo[modelo['total_compras']>0])

(65315, 7091, 2341681, 4293)

In [16]:
modelo=pd.merge(modelo, training, on='person', how='left')

In [17]:
modelo.head()

,person,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,6,6,0,3,3,0,0,0,0,0,0.0
1,00091926,22,448,448,47,2,2,1,0,0,0,0,NaN
2,00091a7a,1,10,0,0,0,0,0,0,0,0,0,NaN
3,000ba417,3,206,206,138,6,6,3,1,1,1,0,NaN
4,000c79fe,1,17,17,17,1,1,1,0,0,0,0,0.0


In [18]:
a_predecir=modelo[modelo['label'].isnull()]
a_predecir.head()

,person,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
1,00091926,22,448,448,47,2,2,1,0,0,0,0,NaN
2,00091a7a,1,10,0,0,0,0,0,0,0,0,0,NaN
3,000ba417,3,206,206,138,6,6,3,1,1,1,0,NaN
5,000e4d9e,9,411,411,112,1,1,0,0,0,0,0,NaN
6,000e619d,4,68,68,0,1,1,0,0,0,0,0,NaN


In [19]:
modelo.drop(modelo[modelo['label'].isnull()].index,inplace=True)
modelo['label']=modelo['label'].astype(int)
modelo.head()

,person,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
0,0008ed71,1,6,6,0,3,3,0,0,0,0,0,0
4,000c79fe,1,17,17,17,1,1,1,0,0,0,0,0
10,001802e4,1,19,19,19,1,1,1,0,0,0,0,0
12,0019e639,14,471,290,0,15,4,0,2,1,0,0,0
14,001b0bf9,1,7,7,0,2,2,0,1,1,0,0,0


In [20]:
len(modelo[modelo['label']==1])

980

In [21]:
len(modelo[modelo['label']==0]) 

18434

In [22]:
len(modelo[modelo['label']==1])*100/len(modelo['label']),len(modelo[modelo['label']==0])*100/len(modelo['label'])

(5.047903574739879, 94.95209642526012)

In [23]:
modelo[modelo['label']==1].head(15)

,person,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes,label
31,003476c0,4,43,14,0,0,0,0,0,0,0,0,1
38,003d4cac,3,8,0,0,1,0,0,0,0,0,0,1
39,003dd965,1,5,5,0,0,0,0,0,0,0,0,1
43,0042421c,2,22,0,0,0,0,0,0,0,0,0,1
57,006671fd,1,18,0,0,0,0,0,0,0,0,0,1
178,0143d796,1,24,24,24,1,1,1,0,0,0,0,1
272,01ed2afc,1,7,7,0,0,0,0,0,0,0,0,1
358,02875920,4,29,22,17,0,0,0,0,0,0,0,1
501,03754d73,6,37,9,9,0,0,0,0,0,0,0,1
559,03c99f07,2,43,43,43,0,0,0,0,0,0,0,1


## MACHINE LEARNING

In [24]:
##Ahora empieza ML

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics

X = modelo.drop(['person','label'], axis=1)
y = modelo['label']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [26]:
len(X_train),len(X_test),len(y_train),len(y_test)

(14560, 4854, 14560, 4854)

In [27]:
X_train.head()

,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes
34844,6,85,85,2,1,1,0,1,1,0,0
8115,2,30,12,12,1,1,1,0,0,0,0
4105,2,14,14,3,1,1,0,0,0,0,0
26513,1,16,16,16,1,1,1,0,0,0,0
22292,1,6,6,0,1,1,0,0,0,0,0


In [28]:
y_train.head()

34844    0
8115     0
4105     0
26513    0
22292    0
Name: label, dtype: int32

In [29]:
X_test.head()

,dias_distintos,total_visitas_usuario,visitas_ultimo_mes,visitas_ultima_semana,total_checkout,checkouts_ultimo_mes,checkouts_ultima_semana,total_compras,compras_ultimo_mes,compras_ultima_semana,subscripciones_ultimo_mes
3693,1,4,4,4,1,1,1,0,0,0,0
20169,2,25,25,20,1,1,1,0,0,0,0
29041,1,15,15,15,2,2,2,0,0,0,0
25040,1,36,36,36,0,0,0,0,0,0,0
27334,2,35,35,5,0,0,0,0,0,0,0


In [30]:
y_test.head()

3693     0
20169    0
29041    0
25040    0
27334    1
Name: label, dtype: int32

In [31]:
len(y_test[y_test == 1]),len(y_test[y_test == 0])

(258, 4596)

## KNN

In [32]:
from sklearn.neighbors import KNeighborsClassifier
my_classifier= KNeighborsClassifier()
my_classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [33]:
predictions= my_classifier.predict(X_test)
accuracy_score(y_test,predictions)

0.9443757725587144

In [34]:
my_matrix = metrics.confusion_matrix(y_test, predictions)
my_matrix

array([[4575,   21],
       [ 249,    9]], dtype=int64)

## DECISION TREE

In [35]:
from sklearn import tree
my_classifier2= tree.DecisionTreeClassifier()
my_classifier2.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [36]:
predictions2= my_classifier2.predict(X_test)
accuracy_score(y_test,predictions2)

0.915533580552122

In [37]:
my_matrix2 = metrics.confusion_matrix(y_test, predictions2)
my_matrix2

array([[4404,  192],
       [ 218,   40]], dtype=int64)

## RANDOM FOREST

In [38]:
from sklearn.ensemble import RandomForestClassifier
my_classifier3 = RandomForestClassifier(n_estimators=100)
my_classifier3.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
predictions3=my_classifier3.predict(X_test)
accuracy_score(y_test, predictions3)

0.9381953028430161

In [40]:
my_matrix3 = metrics.confusion_matrix(y_test, predictions3)
my_matrix3

array([[4538,   58],
       [ 242,   16]], dtype=int64)

## Logistic regression

In [41]:
from sklearn.linear_model import LogisticRegression
my_classifier4 = LogisticRegression()
my_classifier4.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
predictions4 = my_classifier4.predict(X_test)
accuracy_score(y_test, predictions4)

0.945817882159044

In [43]:
my_matrix4 = metrics.confusion_matrix(y_test, predictions4)
my_matrix4

array([[4589,    7],
       [ 256,    2]], dtype=int64)

## Neuronal Network

In [44]:
from sklearn.neural_network import MLPClassifier
my_classifier5 = MLPClassifier(max_iter=100000)
my_classifier5.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [45]:
predictions5 = my_classifier5.predict(X_test)
accuracy_score(y_test, predictions5)

0.9470539761021838

In [46]:
my_matrix5 = metrics.confusion_matrix(y_test, predictions5)
my_matrix5

array([[4596,    0],
       [ 257,    1]], dtype=int64)

## Naive Bayes

In [151]:
from sklearn.naive_bayes import GaussianNB
my_classifier6 = GaussianNB()
my_classifier6.fit(X_train,y_train)

GaussianNB(priors=None)

In [152]:
predictions6 = my_classifier6.predict(X_test)
accuracy_score(y_test, predictions6)

0.8939019365471775

In [153]:
metrics.confusion_matrix(y_test, predictions6)

array([[4298,  298],
       [ 217,   41]], dtype=int64)

## Voting Classifier

In [166]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(solver='saga', multi_class='multinomial', random_state=1)
clf2 = RandomForestClassifier(n_estimators=1000,random_state=1,n_jobs =-1)
clf3 = GaussianNB(priors=[0.05,0.95])
clf4= MLPClassifier()
clf5= tree.DecisionTreeClassifier()
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('mlp',clf4) , ('dt',clf5)], voting='hard', n_jobs =-1)
eclf1 = eclf1.fit(X, y)
predictions = eclf1.predict(X_test)
accuracy_score(y_test, predictions)

d:\programas\python\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9600329625051504

In [167]:
metrics.confusion_matrix(y_test, predictions)

array([[4596,    0],
       [ 194,   64]], dtype=int64)